# Введение в DL

В этом ноутбуке я пару архитектур нейросетей на датасете Mnist, который подгружу из torchvision. Это введение в обучение нейросетей. Основные задачи:
1. Понять, что и откуда нужно импортировать
2. Написать хоть какой-то рабочий код, успешно решающий поставленную (хоть и простую) задачу
3. Приобрести опыт в создании собственных архитектур нейросетей.

## Загрузка и подготовка датасета MNIST

В этом ноутбуке решается задача многоклассовой классификации. Датасет состоит из пар (изображение, класс), где:
* изображение --- чёрно-белое изображение, содержащее какую-то одну рукописную цифру (от 0 до 9)
* класс --- целевая переменная, представляющая собой целое число от 0 до 9, соответствующее цифре на изображении

Задачи:
1. Загрузить датасет
2. Проанализировать размер датасета, размерность данных
3. Подготовить вариант датасета для подачи на вход полносвязной нейросети
4. Также подготовить вариант датасета для подачи на вход свёрточной нейросети

Итак, загрузим датасет:

In [40]:
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

tensor_transform = transforms.ToTensor()

train_set = MNIST(root='./dataset', train=True, transform=tensor_transform, download=True)
test_set = MNIST(root='./dataset', train=False, transform=tensor_transform, download=True)

Посмотрим на характеристики загруженного датасета:

In [41]:
print(f'Тип загруженного из torchvision файла - {type(train_set)}')
print(f'Размерность изображения {tuple(train_set[0][0].size())}. Его тип - {type(train_set[0][0])}')
print(f'Тип целевой метки = {type(train_set[0][1])}')

Тип загруженного из torchvision файла - <class 'torchvision.datasets.mnist.MNIST'>
Размерность изображения (1, 28, 28). Его тип - <class 'torch.Tensor'>
Тип целевой метки = <class 'int'>


Подготовим части датасета для обучения и тестирования. Так как это самый первый ноутбук, то мы сделаем неправильную вещь: возьмём тестовую часть датасета в качестве и тестовой, и валидационной.

In [54]:
from torch.utils.data import DataLoader

bs = 100

train_loader = DataLoader(dataset=train_set, batch_size=bs, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=bs, shuffle=True)

## Создание и обучение полносвязной нейросети

Подготовим архитектуру полносвязной сети:
* Для начала необходимо изменить размерность изображения, вытянув его из 4-мерного вектора размерности `(bs, 1, 28, 28)` в двумерный вектор `(bs, 28*28)`
* После чего добавим несколько полносвязных слоёв с функциями активации LeakyReLU
* На последнем слое вместо LeackyReLU используем Softmax, потому что решается задача задача многоклассовой классификации

In [70]:
import torch.nn as nn
import torch.nn.functional as F


class FCNet(nn.Module):
    def __init__(self):
        super(FCNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = x.view(-1, 28*28) # Вытянем 4-мерный вектор в 2-мерный
        
        # Первый скрытый слой с функцией активации
        x = self.fc1(x)
        x = F.leaky_relu(x)
    
        # Второй скрытый слой с функцией активации
        x = self.fc2(x)
        x = F.leaky_relu(x)

        # Третий скрытый слой с функцией активации
        x = self.fc3(x)
        x = F.leaky_relu(x)
        
        # Выходной слой
        x = self.fc4(x)
        logits = F.softmax(x)
        
        return logits    

Теперь напишем функцию для обучения нашей сети:
* в качестве оптимизатора я буду использовать дефолтный Adam с `learning_rate = 3e-4`
* так как решается задача классификации, то лосс-функцией будет кросс-энтропия
* обучение будет длиться 20 эпох
* на каждой эпохе будет выводиться информация о значении лосс-функции

In [79]:
import torch
import torch.optim as optim
# from torch.autograd import Variable

def train_model(model, train_loader, valid_loader, epochs=10):

    # Использование GPU, если таковая имеется
    use_cuda = False # torch.cuda.is_available()
    if use_cuda:
        model.cuda()
    
    # Настраиваю оптимизатор и лосс-функцию
    optimizer = optim.Adam(model.parameters(), lr=3e-4)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        # one-epoch training
        losses = []
        for x, target in train_loader:
            optimizer.zero_grad()
            if use_cuda:
                x, target = x.cuda(), target.cuda()
            # x, target = Variable(x), Variable(target)
            
            predicted = model(x)
            loss = criterion(predicted, target)
            losses.append(loss.data)
            
            loss.backward()
            optimizer.step()
        train_loss = sum(losses) / len(losses)
        
        # one-epoch validation
        losses = []
        for x, target in valid_loader:
            if use_cuda:
                x, target = x.cuda(), target.cuda()
            
            predicted = model(x)
            loss = criterion(predicted, target)
            losses.append(loss.data)
            
        valid_loss = sum(losses) / len(losses)
        
        print("epoch{:3d}: train_loss == {:5.2f}, valid_loss == {:5.2f}".format(epoch, train_loss, valid_loss))

In [83]:
model = FCNet()
train_model(model, train_loader, test_loader)

C:\Users\02031\anaconda3\envs\torch-cuda\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch  0: train_loss ==  1.65, valid_loss ==  1.55
epoch  1: train_loss ==  1.54, valid_loss ==  1.53
epoch  2: train_loss ==  1.52, valid_loss ==  1.51
epoch  3: train_loss ==  1.51, valid_loss ==  1.51
epoch  4: train_loss ==  1.50, valid_loss ==  1.50
epoch  5: train_loss ==  1.49, valid_loss ==  1.50
epoch  6: train_loss ==  1.49, valid_loss ==  1.49
epoch  7: train_loss ==  1.49, valid_loss ==  1.49
epoch  8: train_loss ==  1.48, valid_loss ==  1.49
epoch  9: train_loss ==  1.48, valid_loss ==  1.49


Модель обучена!

Теперь напишем функцию для тестирования полученной модели. В качестве оценки качества работы модели будет выступать точность классификации.

In [91]:
def test_model(model, train_loader, test_loader):
    use_cuda = False
    if use_cuda:
        model.cuda()
        
    def calculate_accuracy(model, loader):
        correct_n = 0
        total_n = 0

        for x, target in loader:
            if use_cuda:
                x, target = x.cuda(), target.cuda()
            predicted = model(x)
            _, predicted_labels = torch.max(predicted.data, 1)
            total_n += target.data.size()[0]
            correct_n += (predicted_labels == target).sum()
        return correct_n / total_n
    
    train_accuracy = calculate_accuracy(model, train_loader)
    test_accuracy = calculate_accuracy(model, test_loader)
    
    print("Total train accuracy score is {:2.2f}%".format(train_accuracy * 100))
    print("Total test accuracy score is {:2.2f}%".format(test_accuracy * 100))

In [92]:
test_model(model, train_loader, test_loader)

C:\Users\02031\anaconda3\envs\torch-cuda\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Total train accuracy score is 98.38%
Total test accuracy score is 97.35%


Неплохие результаты! Точность классификации полносвязной модели 97.35%

Попробуем теперь применить к этой задаче свёрточную архитектуру.

## Создание и обучение свёрточной нейросети

## TODO: сделать код этого раздела. Добиться качества выше, чем у полносвязной нейросети

Особенности архитектуры:
* Сеть разделена на блоки. Каждый блок состоит из свёрточного слоя, функции активации и аггрегационного слоя (MaxPooling)
* После пары свёрточных блоков будет пара полносвязных слоёв

In [129]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=2)

        self.fc1 = nn.Linear(4*4*64, 128)
        self.fc2 = nn.Linear(128, 10)
        
        self.pool = nn.MaxPool2d(kernel_size=2)
    
    def forward(self, x):

        # Первый свёрточный блок
        x = self.conv1(x)
        x = F.leaky_relu(x)
        x = self.pool(x)
        # print(x.size())

        # Второй свёрточный блок
        x = self.conv2(x)
        x = F.leaky_relu(x)
        x = self.pool(x)
        # print(x.size())
        
        # Третий свёрточный блок
        x = self.conv3(x)
        x = F.leaky_relu(x)
        x = self.pool(x)
        # print(x.size())

        # Вытягивание 4-мерного тензора в 2-мерный
        sz = x.data.size()
        x = x.view(-1, sz[1] * sz[2] * sz[3])

        # Первый полносвязный блок
        x = self.fc1(x)
        x = F.leaky_relu(x)
        
        # Второй полносвязный блок
        x = self.fc2(x)
        logits = F.softmax(x)
        
        return x

Обучим модель со свёрточной архитектурой:

In [130]:
conv_model = ConvNet()
train_model(conv_model, train_loader, test_loader)

C:\Users\02031\anaconda3\envs\torch-cuda\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch  0: train_loss ==  0.52, valid_loss ==  0.14
epoch  1: train_loss ==  0.12, valid_loss ==  0.08
epoch  2: train_loss ==  0.08, valid_loss ==  0.06
epoch  3: train_loss ==  0.06, valid_loss ==  0.04
epoch  4: train_loss ==  0.05, valid_loss ==  0.05
epoch  5: train_loss ==  0.05, valid_loss ==  0.04
epoch  6: train_loss ==  0.04, valid_loss ==  0.04
epoch  7: train_loss ==  0.04, valid_loss ==  0.04
epoch  8: train_loss ==  0.03, valid_loss ==  0.04
epoch  9: train_loss ==  0.03, valid_loss ==  0.03


In [131]:
test_model(conv_model, train_loader, test_loader)

C:\Users\02031\anaconda3\envs\torch-cuda\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Total train accuracy score is 99.38%
Total test accuracy score is 99.01%


Отлично! Точность на тестовой выборке 99.01%, что больше почти на 2% в сравнении с полносвязной моделью.

Сравним ещё число параметров в получившихся моделях:

In [136]:
print("Число параметров в полносвязной сети равно {}".format(sum(p.numel() for p in model.parameters())))
print("Число параметров в свёрточной сети равно   {}".format(sum(p.numel() for p in conv_model.parameters())))

Число параметров в полносвязной сети равно 567434
Число параметров в свёрточной сети равно   155786


Получилось, что число параметров в свёрточной сети почти в 4 раза меньше, чем в полносвязной. 

Из этого можно сделать **вывод**: свёрточные сети позволяют решать задачи обработки изображений с привлечением меньшего числа параметров, чем полносвязные сети. 

В данном конкретном случае свёрточная сеть имеет более высокую точность при значительно более низком числе параметров